# Lab 1: Information Retrieval

__Students:__ 

### Crawling



a) Get the webpage content by using functions in 
__[urllib module](https://docs.python.org/3/library/urllib.html#module-urllib)__.

Other libraries are also fine to achieve the crawling.

e.g. scrapy, beautifulsoup... 

In [5]:
url = "https://play.google.com/store/apps/top"
shortUrl =  "https://play.google.com"

from urllib.request import urlopen
import urllib
import re , numpy

contents = urllib.request.urlopen(url).read().decode('utf8') 


b) Get app url by regular expression using functions from __[re module](https://docs.python.org/3/library/re.html?highlight=re#module-re)__.

A useful online regular expression check.
__[Check your regular expression first](https://regex101.com)__.

In [6]:
mylist = numpy.unique([re.findall(pattern = "href=\"/store/apps/details.*?\"",
                                  string =contents)]) #Getting the link reference
wholeUrl = [str(shortUrl + link[6:-1] + "&hl=en")  for link in mylist]
##Doing the same for each link to get more links
contents2 = [urllib.request.urlopen(link).read().decode('utf8') for link in wholeUrl]

newlist2 =[]
for link in contents2:
    newlist2= newlist2 + re.findall(pattern = "href=\"/store/apps/details.*?\"",
                                    string =link)   #Getting the link reference

newlist2 = numpy.unique(newlist2)
wholeUrl2 = [str(shortUrl + link[6:-1] + "&hl=en")  for link in newlist2 
             if not "reviewId" in link]
wholeUrl2 = numpy.unique(wholeUrl2)

c) Access specific webpage to get description of each app and then store the description in files.

In [7]:
appname =[]
descriptionRes=[]
for link in wholeUrl2[0:1500]:
    with urlopen(link) as url:
        descriptionRes = descriptionRes + re.findall(pattern = "itemprop=\"description.*?\">.*?<div jsname=\".*?\">.*?</div>", string = url.read().decode('utf8'))
    with urlopen(link) as url:    
        appname= appname + re.findall("<title id=\"main-title\">.*? - Android", string = url.read().decode('utf8'))

cleanDes= [link[58:-6] for link in descriptionRes]
appname = [link[23:-10] for link in appname]


### Construct Inverted file index (Vector Model)



d) Preprocess text using NLP techniques from __[nltk module](http://www.nltk.org/py-modindex.html)__.

Using nltk.download(ID) to get the corpora if it is not downloaded before. __[nltk corpora](http://www.nltk.org/nltk_data/)__

In [23]:
import numpy as np

def tf(documents):
    WordList =[]
    
    ##Creating the list of words of all documents being WordList
    for document in documents:
        splitWords = document.split(" ")
        for Word in splitWords:
            if Word not in WordList:
                WordList.append(Word)
    
    ##Matrix with colnames(Documents), rownames = Words            
    tfperdoc = np.zeros([len(WordList), len(documents)])
    for i in range(len(documents)): #### CHange from here
        splitWords = documents[i].split(" ")
        for Word in splitWords:
            tfperdoc[WordList.index(Word), i] +=1
    ##Dividing by the Columnsum and then divide by the larger term in the column
    for column in range(len(documents)):
        tfperdoc [:,column] = tfperdoc [:,column]/np.sum(tfperdoc [:,column])
        tfperdoc [:,column] = tfperdoc [:,column]/np.amax(tfperdoc [:,column])

    return [tfperdoc, WordList]




def idf(tfres):
    #Number of documents in U that contain t
    nwords = tfres.shape[0]
    ndocs = tfres.shape[1]
    wordDoc = np.zeros([nwords, ndocs])
    WordCount = np.zeros([nwords])
    for i in range(nwords):
        for j in range(ndocs):
            if(tfres[i,j]==0):
                wordDoc[i,j]=0
            else:    
                wordDoc[i,j]=1
    for row in range(nwords):
        WordCount[row] = np.sum(wordDoc[row,:])

    for number in range(nwords):
        if WordCount[number] == 0:
            WordCount[number] = 0
        else:
            WordCount[number] = ndocs/WordCount[number]
            WordCount[number] = np.log(WordCount[number])
    return WordCount


def tfidf(tf, idf):
    ndocs = tf.shape[1]
    for column in range(ndocs):
        tf[:,column] = tf[:,column]*idf
        
    return(tf)



def tfidfquery(query, Matwordlist, tf):
    
    queryWordList =[]
    queryWordCount =[]
    ##Creating the list of words with the query being WordList
    
    splitWords = query.split(" ")
    for Word in splitWords:
        if Word not in queryWordList:
            queryWordList.append(Word)
            queryWordCount.append(1)
        else: 
            queryWordCount[queryWordList.index(Word)] +=1
    
    queryWordList =np.asarray(queryWordList)
    queryWordCount =np.asarray(queryWordCount)
    
    ## Matrix of length of vectorwords
    Matrix_tf = np.zeros([len(queryWordList),tf.shape[1]])
    for i in range(len(queryWordList)):
        if queryWordList[i] in Matwordlist:
            Matrix_tf[i,] = tf[Matwordlist.index(queryWordList[i]),] 
    ##Dividing by the Columnsum and then divide by the larger term in the column
    if np.array_equal(Matrix_tf ,np.zeros([len(queryWordList),tf.shape[1]])):
        return "No match found"
    else:
        queryWordCount  = queryWordCount/np.sum(queryWordCount)
        queryWordCount = queryWordCount/np.amax(queryWordCount)  #query word = tf now
    ##Total tf
    Alltf = np.c_[Matrix_tf, queryWordCount] # insert values before column 3
    ###Doing idf    
    Allidf= idf(Alltf)
    ##tfidif =(0.5+0.5tf)*idf
    tfidfres = 0.5*Matrix_tf + 0.5*tfidf(Matrix_tf,Allidf)
    return [tfidfres, queryWordCount.reshape(-1,1)]



In [32]:
from nltk.corpus import stopwords, words
from nltk.tokenize import wordpunct_tokenize

clean = [re.sub(r'[^a-zA-Z0-9\[\]]',' ' ,link).lower() for link in cleanDes] #Clean and tokenize 

###Part 2
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords, words
from nltk.tokenize import TweetTokenizer
import re

def remove_nonalphanum(words):
    return [re.sub(r'[^a-zA-Z0-9\[\]]',' ' , word) for word in words]

def tokenize(words):
    tknzr = TweetTokenizer()
    return [tknzr.tokenize(word) for word in words]

def lowercase(words):
    return [[word.lower() for word in lines] for lines in words]

def stemming(words):
    st = LancasterStemmer()
    return [[st.stem(word) for word in lines] for lines in words]

def stoppingwords(wordes):
    stop= set(stopwords.words('english'))
    stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', 'br']) # remove it if you need punctuation 
    return [[word for word in lines if word not in stop] for lines in wordes]

def checkLanguage(data):
    checkWords = set(words.words())
    return [[word for word in lines if word in checkWords] for lines in data]
    
def joiner(words):
    return [" ".join(word) for word in words] 

def processdata(data):
    functionList = [remove_nonalphanum, tokenize, lowercase, stoppingwords, checkLanguage ,stemming , joiner]
    for function in functionList:
        data = function(data)
    return data




########################################################################3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def QueryProcess(query, NtopResults, dictionary):
    Description = list(dictionary.values())
    appnames = list(dictionary.keys())
    
    k = NtopResults # Number of results to show
    #####################   Matrix numerical transformation 
    Result = processdata(Description) ##Processing data calling function from part2
    tf1=tf(Result)
    idf1= idf(tf1[0])
    tfidfMatrix=tfidf(tf1[0], idf1)       
    
    #Query numerical transformation
    query = processdata([query])[0]###Processing query calling function from part2
    tfidfquest=tfidfquery(query, tf1[1], tf1[0])    
    
    ###Similarity comparison between matrix and query
    search_result = cosine_similarity( tfidfquest[0].transpose(),tfidfquest[1].transpose()).transpose()
    
    SearchNiceArray= search_result.reshape(1,search_result .size)[0]
    kTopIndex = SearchNiceArray.argsort(axis = 0 )[::-1][:k].flatten()
    print([SearchNiceArray[i] for i in kTopIndex ])
    k_recomendations = [appnames[i] for i in kTopIndex]
    return k_recomendations 


...)Compute tdidf using functions from __[scikit-learn module](http://scikit-learn.org/stable/modules/classes.html)__.

eg. TfidfVectorizer is used for converting a collection of raw documents to a matrix of TF-IDF features.

In [33]:
Mydict=dict(zip(appname, clean))





### Query Process

eg. "Dragon, Control, hero, running"

eg. "The hero controls the dragon to run."



In [41]:
myquery1= "dating with people"
myquery2= "Work out fitness"
myquery3= "Cooking food for old people"
myquery4= "wrk ot fitnss"
myquery5= "Capture monster with a ball"

numberRecommendations= 10

print(QueryProcess(myquery1, numberRecommendations, Mydict))



[0.99943677616527915, 0.99787924279608409, 0.99752227309768959, 0.99080533186017461, 0.9908053318601745, 0.9908053318601745, 0.97940201123711323, 0.97940201123711323, 0.97940201123711323, 0.97940201123711323]
['LOVOO', 'Paktor: Meet New People', 'DOWN Dating: Match, Chat, Date', 'Driving Theory Test for Cars 2018', 'OkCupid Dating', 'MyPlate Calorie Tracker', 'Saudi Driving Test - Dallah', 'Calorie Counter - EasyFit free', 'Jaumo Flirt Chat &amp; Dating', 'Summoners War']


It can be seen that it is capturing the "dating" word because it prints out relevant apps

In [35]:
print(QueryProcess(myquery2, numberRecommendations, Mydict))

[0.9988337856652223, 0.98847592334783863, 0.9884759233478384, 0.96051567714137609, 0.93613862503187883, 0.90750634270384523, 0.90750634270384511, 0.90750634270384511, 0.907506342703845, 0.907506342703845]
['Sandbox Number Coloring Book - Color By Number', 'Lose It! - Calorie Counter', 'StrongLifts 5x5 Workout Gym Log &amp; Personal Trainer', 'Calorie Counter - MyNetDiary', 'JEFIT: Workout Tracker, Gym Log &amp; Personal Trainer', 'Abs workout A6W', 'myMail – Email for Hotmail, Gmail and Outlook Mail', 'Fire Emblem Heroes', '30 Day Ab Challenge', '100 GB Free Cloud Drive from Degoo']


While the first results are what we are looking for, because work out is separate, it also takes into account matches for "work" leading to results like googledocs. The data process should be improved in order to suggest not only "work" "out" and  workout together.

In [36]:
print(QueryProcess(myquery3, numberRecommendations, Mydict))

[0.830253794045271, 0.79476003826774844, 0.7471311194301018, 0.7068446797289778, 0.70377686166340414, 0.70293071793689754, 0.68871633846212799, 0.68871633846212799, 0.68871633846212776, 0.68871633846212776]
['Village and Farm', 'Toca Life: Office', 'Calorie Counter - MyFitnessPal', 'Wormax.io', 'Farm On!', 'STRIKERS 1945-2', 'WordOn: multiplayer word game', "What's the Difference?", 'Sites in VR', 'Word Search Online']


Here we are evaluating two topics at the same time, but in fact the algorithm should be taking into account ideally the semantic meaning of the sentence (e.g. looking only for cooking food and giving less importance to old people). It can be seen that it finds some links matching to recipies, but others are more random like farming or messenger. Maybe the cosine similarity is low, but since we are not printing it, we don 't know how similar these last matches are. We should set a minimum threshold on our cosine similarity before returning it.

In [37]:
print(QueryProcess(myquery4, numberRecommendations, Mydict))

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['王國征戰', 'اخترع Amino', 'Castle Clash: فريق الشجعان', 'كويز كلاش', 'Discovery and Invention in Hindi', 'N.O.V.A. Legacy', 'Dragon Mania Legends', 'Sniper Fury: Top shooter -fun shooting games - FPS', 'Green Farm 3', 'March of Empires: War of Lords']


This one is trying misspellings: It can be seen that it does not deal with misspellings. Improvements for that should be done.

In [38]:
print(QueryProcess(myquery5, numberRecommendations, Mydict))

[0.81578660436508832, 0.81578660436508832, 0.80824868062472321, 0.77110116631557823, 0.74322764910004158, 0.57735026918962584, 0.57735026918962584, 0.57735026918962584, 0.57735026918962584, 0.57735026918962584]
['March of Empires: War of Lords', 'Golf Star™', 'World War III: European Wars', 'Balls Bounce', 'Toy Defense Fantasy - TD Strategy Game', 'Defenders 2: Tower Defense CCG', 'Basketball Stars', 'New Subway Surf: Bus Hours 2018', 'Geo dash  ball 4 : in 3 world', 'Last Hope TD - Zombie Tower Defense with Heroes']


In this one I was trying to get back the pokemon app. Instead, we got some other apps that to some extent are maybe more  suitable on their description, but that they are less relevant than others in terms of trendiness. Some weigths should be put inside the algorithm to give importance also to this feature. Also, our description may not be fitting pokemon's description.  